In [1]:
import os
import time
import io
import struct
import sys

In [2]:
infile = 'giphy.gif'
os.path.exists(infile)

True

In [3]:
file_obj = open(infile, mode='rb')

In [50]:
file_obj.close()

# Compare signature

In [5]:
sig = file_obj.read(6)
sig

b'GIF89a'

In [6]:
len(sig)

6

In [7]:
type(sig)

bytes

In [8]:
gif87a_sig = b'GIF87a'
gif89a_sig = b'GIF89a'

In [9]:
msg = f'Hi {str(gif89a_sig)}'
print(msg)

Hi b'GIF89a'


In [10]:
type(gif89a_sig)

bytes

In [11]:
gif89a_sig

b'GIF89a'

In [12]:
sig == gif89a_sig

True

In [13]:
sig == gif87a_sig

False

# Screen Descriptor

In [10]:
screen_desc = file_obj.read(7)

In [15]:
len(screen_desc)

7

In [16]:
type(screen_desc)

bytes

In [11]:
width = struct.unpack('<h', screen_desc[0:2])[0]
height = struct.unpack('<h', screen_desc[2:4])[0]
packed_field = screen_desc[4]
background_color_index = screen_desc[5]
# Pixel Aspect Ratio is normal not used because we've known the width and height.
# pixel_aspect_ratio = screen_desc[6]

## unpack Packed Fields

In [12]:
packed_field

246

In [13]:
type(packed_field)

int

In [14]:
bin(packed_field)

'0b11110110'

In [15]:
global_color_table_flag = (packed_field & 0b10000000) >> 7
global_color_table_flag, bin(global_color_table_flag)

(1, '0b1')

> Color Resolution - Number of bits per primary color available to the original image, minus 1.

I think that means the number of bits to store a color. E.g. true color (RGB) requires 24 bits (3 bytes). Yeah, it makes no sense to me right now because the maximum value for this field is 7 (3 bits all 1).

> This value represents the size of the entire palette from which the colors in the graphic were selected, not the number of colors actually used in the graphic.

I still have no sense what the specification is talking about. The maximum value for this field is 7 so the maximum size of the palette is 7 something?

> For example, if the value in this field is 3, then the palette of the original image had 4 bits per primary color available to create the image.

Now I am noticing the specification is meaning about `original image`. Are there multiple images out there we need to care about outside the image this GIF file is storing? Even you raise the maximum value to 8, we can't still store a RGB color format in 8 bits.

> This value should be set to indicate the richness of the original palette, even if not every color from the whole palette is available on the source machine.

No, that does help at all. Specification!!!

In [16]:
color_resolution = (packed_field & 0b01110000) >> 4
color_resolution, bin(color_resolution)

(7, '0b111')

> Sort Flag - Indicates whether the Global Color Table is sorted.

Simple enough.

> If the flag is set, the Global Color Table is sorted, in order of decreasing importance. Typically, the order would be descreasing frequency, with most frequent color first. This assists a decoder, with fewer available colors, in choosing the best subset of colors; the decoder may use an initial segment of the table to render the graphic.

Yeah, I think I got that.

In [17]:
sort_flag = (packed_field & 0b00001000) >> 3
sort_flag, bin(sort_flag)

(0, '0b0')

> Size of Global Color Table - If the Global Color Table Flag is set to 1, the value in this field is used to calculate the number of bytes contained in the Global Color Table. To determine that actual size of the color table, raise 2 to [the value of the field + 1].

The size of global table is `(field_value + 1) * 2)` bytes, isn't it? So the maximum size for global palette is 16 bytes. Huh, 16 colors or 16 bytes to store color values?

> Even if there is not Global Color Table specified, set this field according to the above formula so that decoders can choose the best graphics mode to display the stream in.

How the size of the palette related to the graphics mode?

In [18]:
global_color_table_size = (packed_field & 0b00000111) >> 0
global_color_table_size, bin(global_color_table_size)

(6, '0b110')